# Mask-RCNN Model Inference in Amazon SageMaker

This notebook is a step-by-step tutorial on [Mask R-CNN](https://arxiv.org/abs/1703.06870) model inference using [Amazon SageMaker model deployment hosting service](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

To get started, we initialize an Amazon execution role and initialize a `boto3` session to find our AWS region name.

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

session = boto3.session.Session()
aws_region = "ap-southeast-1" #session.region_name
print(f'AWS region:{aws_region}')

SageMaker Execution Role:arn:aws:iam::393782509758:role/service-role/AmazonSageMaker-ExecutionRole-20191115T140457
AWS region:ap-southeast-1


## Build and Push Amazon SageMaker Serving Container Images

For this step, the [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) attached to this notebook instance needs full access to [Amazon ECR service](https://aws.amazon.com/ecr/). If you created this notebook instance using the ```./stack-sm.sh``` script in this repository, the IAM Role attached to this notebook instance is already setup with full access to Amazon ECR service. 

Below, we have a choice of two different models for doing inference:

1. [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN)

2. [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow)

It is recommended that you build and push both Amazon SageMaker <b>serving</b> container images below and use one of the two container images for serving the model from an Amazon SageMaker Endpoint.


### Build and Push TensorPack Faster-RCNN/Mask-RCNN Serving Container Image

Use ```./container-serving/build_tools/build_and_push.sh``` script to build and push the [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) <b>serving</b> container image to Amazon ECR. 

In [2]:
!cat ./container-serving/build_tools/build_and_push.sh

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source $DIR/set_env.sh

# set region
region=
if [ "$#" -eq 1 ]; then
    region=$1
else
    echo "usage: $0 <aws-region>"
    exit 1
fi
  

image=$IMAGE_NAME
tag=$IMAGE_TAG

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    aws ecr create-repository --region ${

Using your *AWS region* as argument, run the cell below.

In [1]:
%%time
! ./container-serving/build_tools/build_and_push.sh {aws_region}


Invalid endpoint: https://api.ecr.{aws_region}.amazonaws.com
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon   95.2MB
Step 1/27 : FROM  763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:1.13-horovod-gpu-py36-cu100-ubuntu16.04-v2.0
 ---> 9c51cff02c92
Step 2/27 : ENV TENSORFLOW_VERSION=1.13.2
 ---> Using cache
 ---> cb59c849dccd
Step 3/27 : ENV HOROVOD_VERSION=0.16.4
 ---> Using cache
 ---> 61cc5be5aea3
Step 4/27 : RUN pip install --upgrade pip
 ---> Using cache
 ---> d21317981669
Step 5/27 : RUN pip install tensorflow-gpu==${TENSORFLOW_VERSION} keras h5py
 ---> Using cache
 ---> d5a4313e4c9f
Step 6/27 : RUN ldconfig /usr/local/cuda-10.0/targets/x86_64-linux/lib/stubs && 

Set ```tensorpack_image``` below to Amazon ECR URI of the <b>serving</b> image you pushed above.

In [ ]:
tensorpack_image =  "393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorpack-serving-sagemaker:tf1.13-tp26664c3"#<amazon-ecr-uri>

### Build and Push AWS Samples Mask R-CNN Serving Container Image
Use ```./container-serving-optimized/build_tools/build_and_push.sh``` script to build and push the [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow) <b>serving</b> container image to Amazon ECR.

In [2]:
!cat ./container-serving-optimized/build_tools/build_and_push.sh

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source $DIR/set_env.sh

# set region
region=
if [ "$#" -eq 1 ]; then
    region=$1
else
    echo "usage: $0 <aws-region>"
    exit 1
fi
  

image=$IMAGE_NAME
tag=$IMAGE_TAG

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    aws ecr create-repository --region ${

Using your *AWS region* as argument, run the cell below.

In [3]:
! $(aws ecr get-login --no-include-email --region ap-southeast-1  --registry-ids 393782509758)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [4]:
%%time
! ./container-serving-optimized/build_tools/build_and_push.sh {aws_region}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  95.22MB
Step 1/28 : FROM 393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/invoice-extraction:tensorflow-base-1.13.1-gpu-py36-ubuntu-16.04
tensorflow-base-1.13.1-gpu-py36-ubuntu-16.04: Pulling from invoice-extraction

760c94fc: Pulling fs layer 
92f3c37b: Pulling fs layer 
e5e7f12e: Pulling fs layer 
74cc00ca: Pulling fs layer 
53113e13: Pulling fs layer 
2edc87fb: Pulling fs layer 
f57a58ef: Pulling fs layer 
aa495279: Pulling fs layer 
63b2d43d: Pulling fs layer 
d887aa10: Pulling fs layer 
aa4e079e: Pulling fs layer 
a698eb87: Pulling fs layer 
2ff0c9a2: Pulling fs layer 
ddc7977b: Pulling fs layer 
16202287: Pulling fs layer 
52b810a

92c2b346: Pull complete  959B/959B8MBBKDigest: sha256:9b8bdfe71e42e7755936d6d86b1eb73875dddbadbb26cf6f7fceb6576d7358e7


Status: Downloaded newer image for 393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/invoice-extraction:tensorflow-base-1.13.1-gpu-py36-ubuntu-16.04
 ---> 678566651313
Step 2/28 : ENV HOROVOD_VERSION=0.18.1
 ---> Running in 9aea39bbdfae
Removing intermediate container 9aea39bbdfae
 ---> 865dbb0812a9
Step 3/28 : RUN pip install --upgrade pip
 ---> Running in 97aa43d72ea5
Requirement already up-to-date: pip in /usr/local/lib/python3.6/site-packages (20.0.2)
Removing intermediate container 97aa43d72ea5
 ---> 3c4dad91033f
Step 4/28 : RUN pip uninstall -y tensorflow tensorboard tensorflow-estimator keras h5py horovod numpy
 ---> Running in c6cbdb90d6ac
Found existing installation: tensorflow 1.13.1
Uninstalling tensorflow-1.13.1:
  Successfully uninstalled tensorflow-1.13.1
Found existing installation: tensorflow-estimator 1.13.0
Uninstalling tensorflow-estimator-1.13.0:
  Successfully uninstalled tensorflow-estimator-1.13.0
Found existing installation: Keras 2.2.4
Uninstalling Keras-2.2.4:

  1500K .......... .......... .......... .......... ..........  1% 26.9M 2m0s
  1550K .......... .......... .......... .......... ..........  1% 22.7M 1m57s
  1600K .......... .......... .......... .......... ..........  1% 82.0M 1m53s
  1650K .......... .......... .......... .......... ..........  1%  146M 1m50s
  1700K .......... .......... .......... .......... ..........  1%  159M 1m47s
  1750K .......... .......... .......... .......... ..........  1%  167M 1m44s
  1800K .......... .......... .......... .......... ..........  1%  155M 1m41s
  1850K .......... .......... .......... .......... ..........  1%  144M 98s
  1900K .......... .......... .......... .......... ..........  1%  145M 96s
  1950K .......... .......... .......... .......... ..........  1%  147M 93s
  2000K .......... .......... .......... .......... ..........  1%  145M 91s
  2050K .......... .......... .......... .......... ..........  1%  158M 89s
  2100K .......... .......... .......... .......... .......... 

  5250K .......... .......... .......... .......... ..........  4%  227K 51s
  5300K .......... .......... .......... .......... ..........  4% 35.2M 50s
  5350K .......... .......... .......... .......... ..........  4%  138M 50s
  5400K .......... .......... .......... .......... ..........  4% 40.5M 49s
  5450K .......... .......... .......... .......... ..........  4% 68.3M 49s
  5500K .......... .......... .......... .......... ..........  4%  151M 49s
  5550K .......... .......... .......... .......... ..........  4%  132M 48s
  5600K .......... .......... .......... .......... ..........  4%  101M 48s
  5650K .......... .......... .......... .......... ..........  4% 70.6M 47s
  5700K .......... .......... .......... .......... ..........  4% 51.3M 47s
  5750K .......... .......... .......... .......... ..........  4% 33.0M 46s
  5800K .......... .......... .......... .......... ..........  4%  164M 46s
  5850K .......... .......... .......... .......... ..........  4%  144M 46s

...

Processing /tensorflow-1.13.0-cp36-cp36m-linux_x86_64.whl
ERROR: sagemaker-containers 2.8.6 has requirement scipy>=1.2.2, but you'll have scipy 1.2.1 which is incompatible.
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
Removing intermediate container fb6febd33910
 ---> edc0373df4ac
Step 6/28 : RUN ldconfig /usr/local/cuda-10.0/targets/x86_64-linux/lib/stubs &&     HOROVOD_GPU_ALLREDUCE=NCCL HOROVOD_WITH_TENSORFLOW=1 pip install --no-cache-dir horovod==${HOROVOD_VERSION} &&     ldconfig
 ---> Running in f5057f85122a
  Created wheel for horovod: filename=horovod-0.18.1-cp36-cp36m-linux_x86_64.whl size=94085099 sha256=54c28285ebac682bdb2ecab5977721132aa05765aaf7670ff49ba5fc405c6d80
  Stored in directory: /tmp/pip-ephem-wheel-cache-aujy01ap/wheels/97/63/83/a67efcfcc96182cf4f5f1ac081c9b6d514abd2bd42a9ba8834
Successfully built horov

ERROR: boto3 1.12.34 has requirement botocore<1.16.0,>=1.15.34, but you'll have botocore 1.12.120 which is incompatible.
ERROR: boto3 1.12.34 has requirement s3transfer<0.4.0,>=0.3.0, but you'll have s3transfer 0.2.1 which is incompatible.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.15.34
    Uninstalling botocore-1.15.34:
      Successfully uninstalled botocore-1.15.34
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.3.3
    Uninstalling s3transfer-0.3.3:
      Successfully uninstalled s3transfer-0.3.3
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.3.1
    Uninstalling PyYAML-5.3.1:
      Successfully uninstalled PyYAML-5.3.1
Removing intermediate container 409b411d8b72
 ---> 0927c5c76879
Step 11/28 : RUN pip install boto3
 ---> Running in 28f7abc570bc
ERROR: awscli 1.16.130 has requirement botocore==1.12.120, but you'll have botocore 1.15.41 which is incompatible.
ERROR: awscli 1.16.130 has req

 ---> Running in fcf1282dd0a0
Removing intermediate container fcf1282dd0a0
 ---> e61e6b90ef7c
Step 23/28 : RUN pip install -e /mask-rcnn-tensorflow/
 ---> Running in 7b8bbee6a47a
Obtaining file:///mask-rcnn-tensorflow
  Running setup.py develop for tensorpack
Removing intermediate container 7b8bbee6a47a
 ---> 0e6fcd3f4b8b
Step 24/28 : RUN pip install flask
 ---> Running in ec094653bf69
Removing intermediate container ec094653bf69
 ---> c2ce979f88a7
Step 25/28 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Running in fd720ce02797
Get:1 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  InRelease
Ign:1 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  InRelease
Get:2 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  Release [574 B]
Get:2 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  Release [574 B]
Get:4 http://security.ubuntu.com/ubuntu xenial-security

Setting up libjbig0:amd64 (2.1-3.1) ...
Setting up libgeoip1:amd64 (1.6.9-1) ...
Setting up libicu55:amd64 (55.1-7ubuntu0.5) ...
Setting up libxml2:amd64 (2.9.3+dfsg1-1ubuntu0.7) ...
Setting up libtiff5:amd64 (4.0.6-1ubuntu0.7) ...
Setting up libvpx3:amd64 (1.5.0-2ubuntu1.1) ...
Setting up libxpm4:amd64 (1:3.5.11-1ubuntu0.16.04.1) ...
Setting up libgd3:amd64 (2.1.1-4ubuntu0.16.04.12) ...
Setting up libxslt1.1:amd64 (1.1.28-2.1ubuntu0.3) ...
Setting up nginx-common (1.10.3-0ubuntu0.16.04.5) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Setting up nginx-core (1.10.3-0ubuntu0.16.04.5) ...
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up nginx (1.10.3-0ubuntu0.16.04.5) ...
Processing triggers for libc-bin (2.23-0ubuntu11) ...
Processing triggers for systemd (229-4ubuntu21.22) ...
Removing intermediate container fd

bf04d78: Pushed   709.3MB/700.3MBtf1.13-tp26664c3: digest: sha256:0e5ea3d93f2724c1c8b600dc3fb40cd9dff4858580bb5cc3a812e350c1320a7f size: 9762
Amazon ECR URI: 393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorpack-serving-sagemaker:tf1.13-tp26664c3
CPU times: user 9.99 s, sys: 1.8 s, total: 11.8 s
Wall time: 7min 7s


 Set ```aws_samples_image``` below to Amazon ECR URI of the <b>serving</b> image you pushed above.

In [5]:
aws_samples_image =  "393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorpack-serving-sagemaker:tf1.13-tp26664c3" #<amazon-ecr-uri>



## Select Serving Container Image
Above, we built and pushed [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) and [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow) serving container images to Amazon ECR. Now we are ready to deploy our trained model to an Amazon SageMaker Endpoint using one of the two container images.

Next, we set ```serving_image``` to either the `tensorpack_image` or the `aws_samples_image` variable you defined above, making sure that the serving container image we set below matches our trained model.

In [ ]:
serving_image =  aws_samples_image # set to tensorpack_image or aws_samples_image  variable (no string quotes)
print(f'serving image: {serving_image}')

## Create Amazon SageMaker Session 
Next, we create a SageMaker session.

In [ ]:
sagemaker_session = sagemaker.session.Session(boto_session=session)

## Define Amazon SageMaker Model
Next, we define an Amazon SageMaker Model that defines the deployed model we will serve from an Amazon SageMaker Endpoint. 

In [ ]:
model_name= 'invoice-extraction-invoice-number-256x256-v4'# Name of the model

This model assumes you are using ResNet-50 pre-trained model weights for the ResNet backbone. If this is not true, please adjust  `PRETRAINED_MODEL` value below. Please ensure that the `s3_model_url` of your trained model used below is consistent with the container `serving_image` you set above.

In [ ]:
s3_model_url =  "s3://smart-invoice/mask-rcnn/sagemaker/output/mask-rcnn-s3-2020-04-03-10-45-02-023-optimize-20/output/model.tar.gz"# Trained Model Amazon S3 URI in the format s3://<your path>/model.tar.gz
serving_container_def = {
    'Image': serving_image,
    'ModelDataUrl': s3_model_url,
    'Mode': 'SingleModel',
    'Environment': { 'SM_MODEL_DIR' : '/opt/ml/model', 
                     'PRETRAINED_MODEL': '/ImageNet-R50-AlignPadding.npz'}
}

create_model_response = sagemaker_session.create_model(name=model_name, 
                                                       role=role, 
                                                       container_defs=serving_container_def)

print(create_model_response)

 Next, we set the name of the Amaozn SageMaker hosted service endpoint configuration.

In [ ]:
endpoint_config_name=f'{model_name}-endpoint-config'
print(endpoint_config_name)

Next, we create the Amazon SageMaker hosted service endpoint configuration that uses one instance of `ml.p3.2xlarge` to serve the model.

In [ ]:
epc = sagemaker_session.create_endpoint_config(
    name=endpoint_config_name, 
    model_name=model_name, 
    initial_instance_count=1, 
    instance_type='ml.g4dn.xlarge')
print(epc)

Next we specify the Amazon SageMaker endpoint name for the endpoint used to serve the model.

In [ ]:
endpoint_name=f'{model_name}-endpoint'
print(endpoint_name)

Next, we create the Amazon SageMaker endpoint using the endpoint configuration we created above.

In [ ]:
ep=sagemaker_session.create_endpoint(endpoint_name=endpoint_name, config_name=epc)
print(ep)

Now that the Amazon SageMaker endpoint is in service, we will use the endpoint to do inference for test images. 

Next, we download [COCO 2017 Test images](http://cocodataset.org/#download).

In [ ]:
# !wget -O ~/test2017.zip http://images.cocodataset.org/zips/test2017.zip

We extract the downloaded COCO 2017 Test images to the home directory.

In [ ]:
#!unzip -q -d ~/ ~/test2017.zip
#!rm  ~/test2017.zip

In [ ]:
!mkdir -p invoice_dataset_400/files

In [ ]:
!aws s3 cp "s3://kredx-ai-resources/invoice-annotation-job/files" "invoice_dataset_400/files" --recursive

In [ ]:
!aws s3 cp "s3://smart-invoice/mask-rcnn/sagemaker/input/invoice_dataset_600" "invoice_dataset_600" --recursive

In [ ]:
!aws s3 cp "s3://kredx-ai-resources/invoice-annotation-job-2/files" "files" --recursive --exclude "*" --include "*.jpg"

Below, we will use the downloaded COCO 2017 Test images to test our deployed Mask R-CNN model. However, in order to visualize the detection results, we need to define some helper functions.

## Visualization Helper Functions
Next, we define a helper function to convert COCO Run Length Encoding (RLE) to a binary image mask. 

The RLE encoding is a dictionary with two keys `counts` and `size`. The `counts` value is a list of counts of run lengths of alternating 0s and 1s for an image binary mask for a specific instance segmentation, with the image is scanned row-wise. The `counts` list  starts with a count of 0s. If the binary mask value at `(0,0)` pixel is 1, then the `counts` list starts with a `0`. The `size` value is a list containing image height and width.

In [ ]:
import numpy as np

def rle_to_binary_mask(rle, img_shape):
    value = 0
    mask_array = []
    for count in rle:
        mask_array.extend([int(value)]*count)
        value = (value + 1) % 2
    
    assert len(mask_array) == img_shape[0]*img_shape[1]
    b_mask = np.array(mask_array, dtype=np.uint8).reshape(img_shape)
    
    return b_mask

Next, we define a helper function for generating random colors for visualizing detection results.

In [ ]:
import colorsys
import random

def random_colors(N, bright=False):
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

Next, we define a helper function to apply an image binary mask for an instance segmentation to the image. Each image binary mask is of the size of the image.

In [ ]:
def apply_mask(image, mask, color, alpha=0.5):
    a_mask = np.stack([mask]*3, axis=2).astype(np.int8)
    for c in range(3):
        image[:, :, c] = np.where(mask == 1, image[:, :, c] *(1 - alpha) + alpha * color[c]*255,image[:, :, c])
    return image

Next, we define a helper function to show the applied detection results.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches

def show_detection_results(img=None,
                            annotations=None):
    """
    img: image numpy array
    annotations: annotations array for image where each annotation is in COCO format
    """
    num_annotations = len(annotations)
    colors = random_colors(num_annotations)
    
    fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
    
    for i, a in enumerate(annotations):
        segm = a['segmentation']
        
        img_shape = tuple(segm['size'])
        rle = segm['counts']
        binary_image_mask = rle_to_binary_mask(rle, img_shape)
        
        bbox = a['bbox']
        category_id = a['category_id']
        category_name = a['category_name']
    
        # select color from random colors
        color = colors[i]

        # Show bounding box
        bbox_x, bbox_y, bbox_w, bbox_h = bbox

        box_patch = patches.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, 
                        linewidth=1,
                        alpha=0.8, linestyle="dashed",
                        edgecolor=color, facecolor='none')
        ax.add_patch(box_patch)
        label = f'{category_name}:{category_id}'
        ax.text(bbox_x, bbox_y + 8, label,
                color='red', size=11, backgroundcolor="none")

        # Show mask
        img = apply_mask(img, binary_image_mask.astype(np.bool), color)

    
    ax.imshow(img.astype(int))
    plt.show()

## Visualize Detection Results
Next, we select a random image from COCO 2017 Test image dataset. After you are done visualizing the detection results for this image, you can come back to the cell below and select your next random image to test.

In [ ]:
import os
import random

test2017_dir="invoice_dataset_600/val2017"
files2017_dir="files"
img_id=random.choice(os.listdir(test2017_dir))
img_local_path = os.path.join(test2017_dir,img_id)
file_img_local_path = os.path.join(files2017_dir,img_id.replace("png","jpg"))
print(img_local_path)
print(file_img_local_path)

Next, we read the image and convert it from BGR color to RGB color format.

In [ ]:
import cv2

img=cv2.imread(img_local_path, cv2.IMREAD_COLOR)
print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

Next, we show the image that we randomly selected.

In [ ]:
fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(img.astype(int))
plt.show()

Next, we invoke the Amazon SageMaker Endpoint to detect objects in the test image that we randomly selected.

This REST API endpoint only accepts HTTP POST requests with `ContentType` set to `application/json`. The content of the POST request must conform to following JSON schema:

`{ 
    "img_id": "YourImageId", 
    "img_data": "Base64 encoded image file content, encoded as utf-8 string" 
 }`

The response of the POST request conforms to following JSON schema:

`{ 
    "annotations": [ 
                    {
                        "bbox": [X, Y, width, height], 
                        "category_id": "class id", 
                        "category_name": "class name", 
                        "segmentation": { "counts": [ run-length-encoding, ], "size": [height, width]} 
                     },
                   ]
 }`

In [ ]:
import boto3 
import base64
import json
client = boto3.client('sagemaker-runtime')
with open(img_local_path, "rb") as image_file:
    img_data = base64.b64encode(image_file.read())
    data = {"img_id": img_id}
    data["img_data"] = img_data.decode('utf-8')
    body=json.dumps(data).encode('utf-8')
    
response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType="application/json",
                                  Accept="application/json",
                                  Body=body)
body=response['Body'].read()
msg=body.decode('utf-8')
data=json.loads(msg)
assert data is not None
annotations = data['annotations']
print(annotations)

The response from the endpoint includes annotations for the detected objects in COCO annotations format. 

Next, we aplly all the detection results to the image. 

In [ ]:
show_detection_results(img, annotations)

In [ ]:
import time
images = os.listdir(test2017_dir)
for img_id in images:
    img_local_path = os.path.join(test2017_dir,img_id)
    img=cv2.imread(img_local_path, cv2.IMREAD_COLOR)
    try:
        print(img.shape)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize((800,800), img)
        cv2.imwrite(img_local_path, img)
        
        with open(img_local_path, "rb") as image_file:
            img_data = base64.b64encode(image_file.read())
            data = {"img_id": img_id}
            data["img_data"] = img_data.decode('utf-8')
            body=json.dumps(data).encode('utf-8')

        response = client.invoke_endpoint(EndpointName=endpoint_name,
                                          ContentType="application/json",
                                          Accept="application/json",
                                          Body=body)
        body=response['Body'].read()
        msg=body.decode('utf-8')
        data=json.loads(msg)
        if data:
            annotations = data['annotations']
            print(annotations)
        show_detection_results(img, annotations)
    except:
        pass

## Delete SageMaker Endpoint, Endpoint Config and Model
If you are done testing, delete the deployed Amazon SageMaker endpoint, endpoint config, and the model below. The trained model in S3. bucket is not deleted. If you are not done testing, go back to the section <b>Visualize Detection Results</b> and select another test image.

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name=endpoint_name)
sagemaker_session.delete_endpoint_config(endpoint_config_name=endpoint_config_name)
sagemaker_session.delete_model(model_name=model_name)

In [ ]:
!docker pull "393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/invoice-extraction:tensorflow-base-1.13.1-gpu-py36-ubuntu-16.04"